# Redes generativas

Las Redes Neuronales Recurrentes (RNNs) y sus variantes con celdas controladas, como las Celdas de Memoria a Largo Corto Plazo (LSTMs) y las Unidades Recurrentes Controladas (GRUs), proporcionaron un mecanismo para el modelado del lenguaje, es decir, pueden aprender el orden de las palabras y ofrecer predicciones para la siguiente palabra en una secuencia. Esto nos permite usar las RNNs para **tareas generativas**, como la generación de texto común, la traducción automática e incluso la generación de subtítulos para imágenes.

En la arquitectura de RNN que discutimos en la unidad anterior, cada unidad RNN producía el siguiente estado oculto como salida. Sin embargo, también podemos añadir otra salida a cada unidad recurrente, lo que nos permitiría generar una **secuencia** (que tiene la misma longitud que la secuencia original). Además, podemos usar unidades RNN que no acepten una entrada en cada paso, y simplemente tomen un vector de estado inicial, para luego producir una secuencia de salidas.

En este cuaderno, nos centraremos en modelos generativos simples que nos ayuden a generar texto. Para simplificar, construyamos una **red a nivel de caracteres**, que genera texto letra por letra. Durante el entrenamiento, necesitamos tomar un corpus de texto y dividirlo en secuencias de letras.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

## Construyendo un vocabulario de caracteres

Para construir una red generativa a nivel de caracteres, necesitamos dividir el texto en caracteres individuales en lugar de palabras. La capa `TextVectorization` que hemos estado utilizando antes no puede hacer eso, por lo que tenemos dos opciones:

* Cargar el texto manualmente y realizar la tokenización 'a mano', como se muestra en [este ejemplo oficial de Keras](https://keras.io/examples/generative/lstm_character_level_text_generation/)
* Usar la clase `Tokenizer` para la tokenización a nivel de caracteres.

Optaremos por la segunda opción. `Tokenizer` también se puede usar para tokenizar en palabras, por lo que debería ser fácil cambiar de tokenización a nivel de caracteres a nivel de palabras.

Para realizar la tokenización a nivel de caracteres, necesitamos pasar el parámetro `char_level=True`:


In [2]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

tokenizer = keras.preprocessing.text.Tokenizer(char_level=True,lower=False)
tokenizer.fit_on_texts([x['title'].numpy().decode('utf-8') for x in ds_train])

También queremos usar un token especial para indicar **fin de secuencia**, al que llamaremos `<eos>`. Vamos a añadirlo manualmente al vocabulario:


In [3]:
eos_token = len(tokenizer.word_index)+1
tokenizer.word_index['<eos>'] = eos_token

vocab_size = eos_token + 1

In [4]:
tokenizer.texts_to_sequences(['Hello, world!'])

[[48, 2, 10, 10, 5, 44, 1, 25, 5, 8, 10, 13, 78]]

## Entrenando una RNN generativa para generar títulos

La forma en que entrenaremos una RNN para generar títulos de noticias es la siguiente. En cada paso, tomaremos un título, que será alimentado a una RNN, y para cada carácter de entrada le pediremos a la red que genere el siguiente carácter de salida:

![Imagen que muestra un ejemplo de generación de RNN de la palabra 'HELLO'.](../../../../../lessons/5-NLP/17-GenerativeNetworks/images/rnn-generate.png)

Para el último carácter de nuestra secuencia, le pediremos a la red que genere el token `<eos>`.

La principal diferencia con la RNN generativa que estamos utilizando aquí es que tomaremos una salida de cada paso de la RNN, y no solo de la celda final. Esto se puede lograr especificando el parámetro `return_sequences` en la celda de la RNN.

Así, durante el entrenamiento, una entrada para la red sería una secuencia de caracteres codificados de cierta longitud, y una salida sería una secuencia de la misma longitud, pero desplazada por un elemento y terminada con `<eos>`. El minibatch consistirá en varias de estas secuencias, y necesitaremos usar **padding** para alinear todas las secuencias.

Vamos a crear funciones que transformen el conjunto de datos por nosotros. Debido a que queremos rellenar las secuencias a nivel de minibatch, primero agruparemos el conjunto de datos llamando `.batch()`, y luego usaremos `map` para realizar la transformación. Por lo tanto, la función de transformación tomará un minibatch completo como parámetro:


In [5]:
def title_batch(x):
    x = [t.numpy().decode('utf-8') for t in x]
    z = tokenizer.texts_to_sequences(x)
    z = tf.keras.preprocessing.sequence.pad_sequences(z)
    return tf.one_hot(z,vocab_size), tf.one_hot(tf.concat([z[:,1:],tf.constant(eos_token,shape=(len(z),1))],axis=1),vocab_size)

Algunas cosas importantes que hacemos aquí:
* Primero extraemos el texto real del tensor de cadenas
* `text_to_sequences` convierte la lista de cadenas en una lista de tensores de enteros
* `pad_sequences` rellena esos tensores hasta su longitud máxima
* Finalmente codificamos en formato one-hot todos los caracteres, y también realizamos el desplazamiento y la adición de `<eos>`. Pronto veremos por qué necesitamos caracteres codificados en formato one-hot.

Sin embargo, esta función es **Pythonic**, es decir, no puede ser traducida automáticamente al gráfico computacional de Tensorflow. Obtendremos errores si intentamos usar esta función directamente en la función `Dataset.map`. Necesitamos encapsular esta llamada Pythonic utilizando el envoltorio `py_function`:


In [6]:
def title_batch_fn(x):
    x = x['title']
    a,b = tf.py_function(title_batch,inp=[x],Tout=(tf.float32,tf.float32))
    return a,b

> **Nota**: Diferenciar entre las funciones de transformación de Python y Tensorflow puede parecer un poco complejo, y podrías estar preguntándote por qué no transformamos el conjunto de datos usando funciones estándar de Python antes de pasarlo a `fit`. Aunque esto definitivamente se puede hacer, usar `Dataset.map` tiene una gran ventaja, ya que la tubería de transformación de datos se ejecuta utilizando el gráfico computacional de Tensorflow, lo que aprovecha las capacidades de cálculo de la GPU y minimiza la necesidad de transferir datos entre la CPU y la GPU.

Ahora podemos construir nuestra red generadora y comenzar el entrenamiento. Puede basarse en cualquier célula recurrente que discutimos en la unidad anterior (simple, LSTM o GRU). En nuestro ejemplo, utilizaremos LSTM.

Dado que la red toma caracteres como entrada y el tamaño del vocabulario es bastante pequeño, no necesitamos una capa de embeddings; la entrada codificada en formato one-hot puede ir directamente a la célula LSTM. La capa de salida sería un clasificador `Dense` que convertirá la salida de LSTM en números de tokens codificados en formato one-hot.

Además, dado que estamos trabajando con secuencias de longitud variable, podemos usar la capa `Masking` para crear una máscara que ignore la parte rellenada de la cadena. Esto no es estrictamente necesario, ya que no estamos muy interesados en todo lo que va más allá del token `<eos>`, pero lo utilizaremos con el propósito de adquirir experiencia con este tipo de capa. El `input_shape` sería `(None, vocab_size)`, donde `None` indica la secuencia de longitud variable, y la forma de salida es también `(None, vocab_size)`, como puedes ver en el `summary`:


In [7]:
model = keras.models.Sequential([
    keras.layers.Masking(input_shape=(None,vocab_size)),
    keras.layers.LSTM(128,return_sequences=True),
    keras.layers.Dense(vocab_size,activation='softmax')
])

model.summary()
model.compile(loss='categorical_crossentropy')

model.fit(ds_train.batch(8).map(title_batch_fn))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, None, 84)          0         
_________________________________________________________________
lstm (LSTM)                  (None, None, 128)         109056    
_________________________________________________________________
dense (Dense)                (None, None, 84)          10836     
Total params: 119,892
Trainable params: 119,892
Non-trainable params: 0
_________________________________________________________________
15000/15000 [==============================] - 229s 15ms/step - loss: 1.5385


## Generando salida

Ahora que hemos entrenado el modelo, queremos usarlo para generar algo de salida. Primero, necesitamos una forma de decodificar texto representado por una secuencia de números de tokens. Para ello, podríamos usar la función `tokenizer.sequences_to_texts`; sin embargo, no funciona bien con la tokenización a nivel de caracteres. Por lo tanto, tomaremos un diccionario de tokens del tokenizer (llamado `word_index`), construiremos un mapa inverso y escribiremos nuestra propia función de decodificación:


In [10]:
reverse_map = {val:key for key, val in tokenizer.word_index.items()}

def decode(x):
    return ''.join([reverse_map[t] for t in x])

Ahora, vamos a generar. Comenzaremos con una cadena `start`, la codificaremos en una secuencia `inp`, y luego en cada paso llamaremos a nuestra red para inferir el siguiente carácter.

La salida de la red `out` es un vector de `vocab_size` elementos que representa las probabilidades de cada token, y podemos encontrar el número del token más probable utilizando `argmax`. Luego, añadimos este carácter a la lista generada de tokens y continuamos con la generación. Este proceso de generar un carácter se repite `size` veces para generar el número requerido de caracteres, y terminamos antes si se encuentra el `eos_token`.


In [12]:
def generate(model,size=100,start='Today '):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            nc = tf.argmax(out)
            if nc==eos_token:
                break
            chars.append(nc.numpy())
            inp = inp+[nc]
        return decode(chars)
    
generate(model)

'Today #39;s lead to strike for the strike for the strike for the strike (AFP)'

## Muestreo de salida durante el entrenamiento

Dado que no tenemos métricas útiles como *precisión*, la única forma de verificar que nuestro modelo está mejorando es **muestreando** cadenas generadas durante el entrenamiento. Para hacerlo, utilizaremos **callbacks**, es decir, funciones que podemos pasar a la función `fit` y que se llamarán periódicamente durante el entrenamiento.


In [13]:
sampling_callback = keras.callbacks.LambdaCallback(
  on_epoch_end = lambda batch, logs: print(generate(model))
)

model.fit(ds_train.batch(8).map(title_batch_fn),callbacks=[sampling_callback],epochs=3)

Epoch 1/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.2703
Today #39;s a lead in the company for the strike
Epoch 2/3
15000/15000 [==============================] - 227s 15ms/step - loss: 1.2057
Today #39;s the Market Service on Security Start (AP)
Epoch 3/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.1752
Today #39;s a line on the strike to start for the start


Este ejemplo ya genera un texto bastante bueno, pero se puede mejorar de varias maneras:

* **Más texto**. Solo hemos utilizado títulos para nuestra tarea, pero podrías experimentar con texto completo. Recuerda que las RNN no son muy buenas manejando secuencias largas, por lo que tiene sentido dividirlas en oraciones más cortas o entrenar siempre con una longitud de secuencia fija de algún valor predefinido `num_chars` (por ejemplo, 256). Podrías intentar modificar el ejemplo anterior para usar esta arquitectura, utilizando el [tutorial oficial de Keras](https://keras.io/examples/generative/lstm_character_level_text_generation/) como inspiración.

* **LSTM multicapa**. Tiene sentido probar con 2 o 3 capas de células LSTM. Como mencionamos en la unidad anterior, cada capa de LSTM extrae ciertos patrones del texto, y en el caso de un generador a nivel de caracteres, podemos esperar que el nivel inferior de LSTM sea responsable de extraer sílabas, y los niveles superiores de palabras y combinaciones de palabras. Esto se puede implementar fácilmente pasando un parámetro de número de capas al constructor de LSTM.

* También podrías experimentar con **unidades GRU** y ver cuáles funcionan mejor, así como con **diferentes tamaños de capas ocultas**. Una capa oculta demasiado grande puede resultar en sobreajuste (por ejemplo, la red aprenderá el texto exacto), y un tamaño más pequeño podría no producir buenos resultados.


## Generación de texto suave y temperatura

En la definición anterior de `generate`, siempre tomábamos el carácter con la mayor probabilidad como el siguiente carácter en el texto generado. Esto daba como resultado que el texto a menudo "ciclaba" entre las mismas secuencias de caracteres una y otra vez, como en este ejemplo:
```
today of the second the company and a second the company ...
```

Sin embargo, si observamos la distribución de probabilidad para el siguiente carácter, podría suceder que la diferencia entre algunas de las probabilidades más altas no sea muy grande, por ejemplo, un carácter puede tener una probabilidad de 0.2, otro de 0.19, etc. Por ejemplo, al buscar el siguiente carácter en la secuencia '*play*', el siguiente carácter podría ser igualmente un espacio o **e** (como en la palabra *player*).

Esto nos lleva a la conclusión de que no siempre es "justo" seleccionar el carácter con mayor probabilidad, ya que elegir el segundo más alto aún podría llevarnos a un texto significativo. Es más sabio **muestrear** caracteres de la distribución de probabilidad proporcionada por la salida de la red.

Este muestreo se puede realizar utilizando la función `np.multinomial`, que implementa la llamada **distribución multinomial**. Una función que implementa esta generación de texto **suave** se define a continuación:


In [33]:
def generate_soft(model,size=100,start='Today ',temperature=1.0):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            probs = tf.exp(tf.math.log(out)/temperature).numpy().astype(np.float64)
            probs = probs/np.sum(probs)
            nc = np.argmax(np.random.multinomial(1,probs,1))
            if nc==eos_token:
                break
            chars.append(nc)
            inp = inp+[nc]
        return decode(chars)

words = ['Today ','On Sunday ','Moscow, ','President ','Little red riding hood ']
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"\n--- Temperature = {i}")
    for j in range(5):
        print(generate_soft(model,size=300,start=words[j],temperature=i))


--- Temperature = 0.3
Today #39;s strike #39; to start at the store return
On Sunday PO to Be Data Profit Up (Reuters)
Moscow, SP wins straight to the Microsoft #39;s control of the space start
President olding of the blast start for the strike to pay &lt;b&gt;...&lt;/b&gt;
Little red riding hood ficed to the spam countered in European &lt;b&gt;...&lt;/b&gt;

--- Temperature = 0.8
Today countie strikes ryder missile faces food market blut
On Sunday collores lose-toppy of sale of Bullment in &lt;b&gt;...&lt;/b&gt;
Moscow, IBM Diffeiting in Afghan Software Hotels (Reuters)
President Ol Luster for Profit Peaced Raised (AP)
Little red riding hood dace on depart talks #39; bank up

--- Temperature = 1.0
Today wits House buiting debate fixes #39; supervice stake again
On Sunday arling digital poaching In for level
Moscow, DS Up 7, Top Proble Protest Caprey Mamarian Strike
President teps help of roubler stepted lessabul-Dhalitics (AFP)
Little red riding hood signs on cash in Carter-youb

---

KeyError: 0

Hemos introducido un parámetro más llamado **temperatura**, que se utiliza para indicar qué tan estrictamente debemos adherirnos a la probabilidad más alta. Si la temperatura es 1.0, hacemos un muestreo multinomial justo, y cuando la temperatura se acerca al infinito, todas las probabilidades se vuelven iguales y seleccionamos el siguiente carácter al azar. En el ejemplo a continuación, podemos observar que el texto se vuelve sin sentido cuando aumentamos demasiado la temperatura, y se asemeja a un texto "cíclico" generado rígidamente cuando se acerca a 0.



---

**Descargo de responsabilidad**:  
Este documento ha sido traducido utilizando el servicio de traducción automática [Co-op Translator](https://github.com/Azure/co-op-translator). Si bien nos esforzamos por garantizar la precisión, tenga en cuenta que las traducciones automatizadas pueden contener errores o imprecisiones. El documento original en su idioma nativo debe considerarse la fuente autorizada. Para información crítica, se recomienda una traducción profesional realizada por humanos. No nos hacemos responsables de malentendidos o interpretaciones erróneas que puedan surgir del uso de esta traducción.
